In [1]:
from neural_network import *

from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

In [2]:
input_size1 = 5
output_size1 = 2
input_size2 = 3
output_size2 = 2

filename = 'model.pt'

NN = Neural_network(input_size1,output_size1,input_size2,output_size2)

NN.load(filename)
NN.eval()

Neural_network(
  (net1): Sequential(
    (0): Linear(in_features=5, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
    (5): Linear(in_features=30, out_features=2, bias=True)
  )
  (net2): Sequential(
    (0): Linear(in_features=10, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
    (5): Linear(in_features=50, out_features=2, bias=True)
  )
)

In [3]:
file_path = "../Data/01_Inicial_v8.xlsx"

D = Data_loader(file_path,seed=0)

In [4]:
inputs = np.zeros((0,input_size1+input_size2))
targets = np.zeros((0,output_size1+output_size2))
preds = np.zeros((0,output_size1+output_size2))

for X, Y_target in D.get_batch2(batch_size=10):
    Y_pred = NN(X)

    X1 = X[:,:input_size1].cpu().numpy()
    X2 = X[:,input_size1:].cpu().numpy()

    Y1_target = Y_target[:,:output_size1].cpu().numpy()
    Y2_target = Y_target[:,output_size1:].cpu().numpy()

    Y1_pred = Y_pred[:,:output_size1].cpu().detach().numpy()
    Y2_pred = Y_pred[:,output_size1:].cpu().detach().numpy()

    X1,X2 = NN.denormalize_X(X1=X1,X2=X2)
    Y1_target,Y2_target = NN.denormalize_Y(Y1=Y1_target,Y2=Y2_target)
    Y1_pred,Y2_pred = NN.denormalize_Y(Y1=Y1_pred,Y2=Y2_pred)

    inputs = np.vstack((inputs,np.hstack((X1,X2))))

    targets = np.vstack((targets,np.hstack((Y1_target,Y2_target))))
    preds = np.vstack((preds,np.hstack((Y1_pred,Y2_pred))))

n_train = inputs.shape[0]

for X, Y_target in D.get_batch2(batch_size=10,validation=True):
    Y_pred = NN(X)

    X1 = X[:,:input_size1].cpu().numpy()
    X2 = X[:,input_size1:].cpu().numpy()

    Y1_target = Y_target[:,:output_size1].cpu().numpy()
    Y2_target = Y_target[:,output_size1:].cpu().numpy()

    Y1_pred = Y_pred[:,:output_size1].cpu().detach().numpy()
    Y2_pred = Y_pred[:,output_size1:].cpu().detach().numpy()

    X1,X2 = NN.denormalize_X(X1=X1,X2=X2)
    Y1_target,Y2_target = NN.denormalize_Y(Y1=Y1_target,Y2=Y2_target)
    Y1_pred,Y2_pred = NN.denormalize_Y(Y1=Y1_pred,Y2=Y2_pred)

    inputs = np.vstack((inputs,np.hstack((X1,X2))))

    targets = np.vstack((targets,np.hstack((Y1_target,Y2_target))))
    preds = np.vstack((preds,np.hstack((Y1_pred,Y2_pred))))

train = np.zeros((inputs.shape[0],1))
train[:n_train,0] = 1



In [5]:

input_names = D.input_columns_DDO + D.input_columns_RBDO
output_names = D.output_columns_DDO + D.output_columns_RBDO
output_names_target = [f'{c}_target' for c in output_names]
output_names_pred = [f'{c}_pred' for c in output_names]

df = pd.DataFrame(np.hstack((inputs,targets,preds,train)), columns=input_names+output_names_target+output_names_pred+['train'])

df.to_csv('output.csv', sep=',', index=False)

In [6]:
df

,c,phi,gamma,HV,H,c_cov,phi_cov,gamma_cov,V1_target,FS_target,V2_target,PF_target,V1_pred,FS_pred,V2_pred,PF_pred,train
0,8.0,28.0,20.0,1.5,20.0,15.000001,26.0,9.0,149.322288,1.276,155.007471,0.178400,166.537044,1.288181,173.294013,0.178433,1.0
1,10.0,20.0,16.0,2.0,10.0,49.999999,30.0,11.0,142.508555,1.414,116.303234,0.142600,150.891003,1.418165,115.290538,0.135637,1.0
2,17.0,27.0,16.0,1.5,30.0,28.000000,19.0,5.0,417.841359,1.431,417.841360,0.026970,447.422246,1.440333,425.909527,0.036252,1.0
3,22.0,23.0,16.0,1.0,30.0,18.000000,16.0,8.0,370.395290,1.091,370.395291,0.280200,324.355501,1.111576,352.996445,0.265427,1.0
4,11.0,21.0,16.0,2.5,20.0,24.000000,15.0,11.0,783.695040,1.439,783.695027,0.005525,767.883185,1.459268,777.315180,0.008436,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,20.0,30.0,18.0,1.5,30.0,34.000000,23.0,10.0,417.841359,1.588,417.841360,0.023610,400.678729,1.550280,383.898037,0.041350,0.0
265,10.0,20.0,15.0,2.5,30.0,23.000000,26.0,3.0,1524.869891,1.260,1623.708052,0.189200,1554.648279,1.319510,1746.292299,0.080352,0.0
266,28.0,25.0,18.0,1.5,20.0,46.000000,21.0,4.0,276.248607,1.703,164.598103,0.040380,276.408219,1.712871,155.184266,0.026071,0.0
267,21.0,20.0,19.0,1.5,20.0,39.000000,18.0,10.0,276.248607,1.285,217.825126,0.130100,251.130782,1.326037,233.894036,0.088341,0.0
